In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
## checking data from website to see if I can use a for loop to iterate over each game

year = pd.read_html('https://www.footballdb.com/games/index.html')

In [3]:
## this is data for the first week

year[0]['Date']

0     09/05/201909/05
1     09/08/201909/08
2     09/08/201909/08
3     09/08/201909/08
4     09/08/201909/08
5     09/08/201909/08
6     09/08/201909/08
7     09/08/201909/08
8     09/08/201909/08
9     09/08/201909/08
10    09/08/201909/08
11    09/08/201909/08
12    09/08/201909/08
13    09/08/201909/08
14    09/09/201909/09
15    09/09/201909/09
Name: Date, dtype: object

In [4]:
## changing the date to match url

year[0]['Date'][0][6:]

'201909/05'

In [5]:
re.sub("[^\d]", "", year[0]['Date'][0][6:])

'20190905'

In [6]:
## writing function to convert all game dates to game id's for one year, returns only regular season games

def get_game_ids(url):

    year = pd.read_html(url)
    game_ids = []
    
    for weeks in year:
        week = []
        count = 1
        for num in range(len(weeks['Date'])):
            game_id = re.sub("[^\d]", "", weeks['Date'][num][6:])
            
            if count == 1 and num != 1:
                old_game_id = game_id
            if num > 0 and old_game_id == game_id:
                count +=1
            else:
                count = 1
            
            if count < 10:
                week.append(game_id + '0' + str(count))
            else:
                week.append(game_id + str(count))
                
        game_ids.append(week)
    
    return game_ids[:17]

In [7]:
## checking to see if the code worked properly

get_game_ids('https://www.footballdb.com/games/index.html')

[['2019090501',
  '2019090801',
  '2019090802',
  '2019090803',
  '2019090804',
  '2019090805',
  '2019090806',
  '2019090807',
  '2019090808',
  '2019090809',
  '2019090810',
  '2019090811',
  '2019090812',
  '2019090813',
  '2019090901',
  '2019090902'],
 ['2019091201',
  '2019091501',
  '2019091502',
  '2019091503',
  '2019091504',
  '2019091505',
  '2019091506',
  '2019091507',
  '2019091508',
  '2019091509',
  '2019091510',
  '2019091511',
  '2019091512',
  '2019091513',
  '2019091514',
  '2019091601'],
 ['2019091901',
  '2019092201',
  '2019092202',
  '2019092203',
  '2019092204',
  '2019092205',
  '2019092206',
  '2019092207',
  '2019092208',
  '2019092209',
  '2019092210',
  '2019092211',
  '2019092212',
  '2019092213',
  '2019092214',
  '2019092301'],
 ['2019092601',
  '2019092901',
  '2019092902',
  '2019092903',
  '2019092904',
  '2019092905',
  '2019092906',
  '2019092907',
  '2019092908',
  '2019092909',
  '2019092910',
  '2019092911',
  '2019092912',
  '2019092913',
  '20

In [8]:
## taking a look at the game data

game_data = pd.read_html('https://www.footballdb.com/games/boxscore.html?gid=2017111601')

In [9]:
## putting score table into a variable to determine winner/loser of match

score = game_data[0]
score

,0,1,2,3,4,5
0,NaN,1,2,3,4,Total
1,Tennessee TitansTennessee (6-4),7,0,10,0,17
2,Pittsburgh SteelersPittsburgh (8-2),10,6,7,17,40


In [10]:
## converting data I want from website and transposing it so columns have features

data_1 = game_data[2].set_index('Unnamed: 0').T

In [11]:
data_2 = game_data[3].set_index('Unnamed: 0').T

In [12]:
data_1

Unnamed: 0,First downs,Rushing,Passing,Penalty,Total Net Yards,Net Yards Rushing,Rushing Plays,Average Gain,Net Yards Passing,Att - Comp - Int,Sacked - Yds Lost,Gross Yards Passing,Avg. Yds/Att
TennesseeTen,17,5,9,3,316,52,21,2.5,264,34-22-4,5-42,306,9.0
PittsburghPit,25,3,19,3,349,65,21,3.1,284,45-30-0,3-15,299,6.6


In [13]:
## drop unnecessary columns

data_1.drop(labels=['Rushing', 'Passing', 'Penalty', 'Att - Comp - Int'], axis=1, inplace=True)
data_1

Unnamed: 0,First downs,Total Net Yards,Net Yards Rushing,Rushing Plays,Average Gain,Net Yards Passing,Sacked - Yds Lost,Gross Yards Passing,Avg. Yds/Att
TennesseeTen,17,316,52,21,2.5,264,5-42,306,9.0
PittsburghPit,25,349,65,21,3.1,284,3-15,299,6.6


In [14]:
## changing sacked - yds lost count the number of sacks only

data_1['Sacked'] = [data_1['Sacked - Yds Lost'][0].split('-')[0], data_1['Sacked - Yds Lost'][1].split('-')[0]]

In [15]:
data_1.drop('Sacked - Yds Lost', axis=1, inplace=True)

In [16]:
## rename column to be more specific

data_1.rename(columns={'Avg. Yds/Att': 'Avg. Yds/Att Passing', 'Average Gain': 'Avg. Yds/Rush'}, inplace=True)

In [17]:
## checking to see if data from first table has been cleaned up

data_1

Unnamed: 0,First downs,Total Net Yards,Net Yards Rushing,Rushing Plays,Avg. Yds/Rush,Net Yards Passing,Gross Yards Passing,Avg. Yds/Att Passing,Sacked
TennesseeTen,17,316,52,21,2.5,264,306,9.0,5
PittsburghPit,25,349,65,21,3.1,284,299,6.6,3


In [18]:
## moving on to cleaning up data from second table

data_2

Unnamed: 0,Punts - Average,Had Blocked,Punt Returns,Kickoff Returns,Interception Returns,Penalties - Yards,Fumbles - Lost,Field Goals,Third Downs,Fourth Downs,Total Plays,Average Gain,Time of Possession
TennesseeTen,3-47.3,0,1-10,5-103,0-0,9-65,0-0,1-2,5-13-38%,1-2-50%,60,5.3,28:20
PittsburghPit,3-50.7,0,2-23,1-21,4-98,7-61,1-0,4-4,4-12-33%,0-1-0%,69,5.1,31:40


In [19]:
## dropping unnecessary columns

data_2.drop(labels=['Punts - Average', 'Had Blocked', 'Punt Returns', 'Kickoff Returns', 'Field Goals', 'Time of Possession'], axis=1, inplace=True)

In [20]:
## changing interception returns column to count interceptions only

data_2['Interceptions'] = [data_2['Interception Returns'][0].split('-')[0], data_2['Interception Returns'][1].split('-')[0]]

In [21]:
## changing fumbles - lost column to count lost fumbles only

data_2['Fumbles Lost'] = [data_2['Fumbles - Lost'][0].split('-')[1], data_2['Fumbles - Lost'][1].split('-')[1]]

In [22]:
## changing penalties - yards column to avg yards lost per penalty. Making if statements so there is no division by zero

v_penalties = int(data_2['Penalties - Yards'][0].split('-')[0])
h_penalties = int(data_2['Penalties - Yards'][1].split('-')[0])

if v_penalties == 0 and h_penalties != 0:
    data_2['Penalty Avg. Yds Lost'] = [0, int(data_2['Penalties - Yards'][1].split('-')[1]) / h_penalties]
elif v_penalties != 0 and h_penalties == 0:
    data_2['Penalty Avg. Yds Lost'] = [int(data_2['Penalties - Yards'][0].split('-')[1]) / v_penalties, 0]
elif v_penalties == 0 and h_penalties == 0:
    data_2['Penalty Avg. Yds Lost'] = [0, 0]
else:
    data_2['Penalty Avg. Yds Lost'] = [int(data_2['Penalties - Yards'][0].split('-')[1]) / v_penalties,
                                       int(data_2['Penalties - Yards'][1].split('-')[1]) / h_penalties]

In [23]:
## changing third and fourth down columns to third/fourth down successful conversion percentage

data_2['Third Down Pct'] = [int(data_2['Third Downs'][0].split('-')[2][:-1])/100, int(data_2['Third Downs'][1].split('-')[2][:-1])/100]
data_2['Fourth Down Pct'] = [int(data_2['Fourth Downs'][0].split('-')[2][:-1])/100, int(data_2['Fourth Downs'][1].split('-')[2][:-1])/100]

In [24]:
## drop unneeded columns

data_2.drop(labels=['Interception Returns', 'Penalties - Yards', 'Fumbles - Lost', 'Third Downs', 'Fourth Downs'], axis=1, inplace=True)

In [25]:
## checking to see if data is cleaned up

data_2

Unnamed: 0,Total Plays,Average Gain,Interceptions,Fumbles Lost,Penalty Avg. Yds Lost,Third Down Pct,Fourth Down Pct
TennesseeTen,60,5.3,0,0,7.222222,0.38,0.5
PittsburghPit,69,5.1,4,0,8.714286,0.33,0.0


In [26]:
## combine data 1 and 2 tables

combined = pd.concat([data_1, data_2], axis=1)
combined

Unnamed: 0,First downs,Total Net Yards,Net Yards Rushing,Rushing Plays,Avg. Yds/Rush,Net Yards Passing,Gross Yards Passing,Avg. Yds/Att Passing,Sacked,Total Plays,Average Gain,Interceptions,Fumbles Lost,Penalty Avg. Yds Lost,Third Down Pct,Fourth Down Pct
TennesseeTen,17,316,52,21,2.5,264,306,9.0,5,60,5.3,0,0,7.222222,0.38,0.5
PittsburghPit,25,349,65,21,3.1,284,299,6.6,3,69,5.1,4,0,8.714286,0.33,0.0


In [27]:
## separating data by home and visitor teams

visitor_columns = []
home_columns = []

for column in combined.columns:
    visitor_columns.append('V_' + column)
    home_columns.append('H_' + column)

In [28]:
## combining game data into one row

game_data = list(combined.iloc[0]) + list(combined.iloc[1])

In [29]:
## changing features from string to int/float

pd.to_numeric(game_data)

array([1.70000000e+01, 3.16000000e+02, 5.20000000e+01, 2.10000000e+01,
       2.50000000e+00, 2.64000000e+02, 3.06000000e+02, 9.00000000e+00,
       5.00000000e+00, 6.00000000e+01, 5.30000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.22222222e+00, 3.80000000e-01, 5.00000000e-01,
       2.50000000e+01, 3.49000000e+02, 6.50000000e+01, 2.10000000e+01,
       3.10000000e+00, 2.84000000e+02, 2.99000000e+02, 6.60000000e+00,
       3.00000000e+00, 6.90000000e+01, 5.10000000e+00, 4.00000000e+00,
       0.00000000e+00, 8.71428571e+00, 3.30000000e-01, 0.00000000e+00])

In [30]:
# transfer game data to dataframe

game_df = pd.DataFrame([game_data], columns=visitor_columns + home_columns)

In [31]:
## check to see if data is correct

game_df

,V_First downs,V_Total Net Yards,V_Net Yards Rushing,V_Rushing Plays,V_Avg. Yds/Rush,V_Net Yards Passing,V_Gross Yards Passing,V_Avg. Yds/Att Passing,V_Sacked,V_Total Plays,...,H_Gross Yards Passing,H_Avg. Yds/Att Passing,H_Sacked,H_Total Plays,H_Average Gain,H_Interceptions,H_Fumbles Lost,H_Penalty Avg. Yds Lost,H_Third Down Pct,H_Fourth Down Pct
0,17,316,52,21,2.5,264,306,9.0,5,60,...,299,6.6,3,69,5.1,4,0,8.714286,0.33,0.0


In [32]:
## adding if team was home or away for game

combined['Home'] = [0, 1]

In [33]:
## adding outcome of game to dataframe, if home team wins outcome = 1, if visitor wins outcome = 0, if tie both team outcomes = 0.5

if int(score[5][1]) > int(score[5][2]):
    game_df['Outcome'] = 0
    combined['Outcome'] = [1, 0]
elif int(score[5][1]) == int(score[5][2]):
    game_df['Outcome'] = 0.5
    combined['Outcome'] = [0.5, 0.5]
else:
    game_df['Outcome'] = 1
    combined['Outcome'] = [0, 1]

In [34]:
game_df

,V_First downs,V_Total Net Yards,V_Net Yards Rushing,V_Rushing Plays,V_Avg. Yds/Rush,V_Net Yards Passing,V_Gross Yards Passing,V_Avg. Yds/Att Passing,V_Sacked,V_Total Plays,...,H_Avg. Yds/Att Passing,H_Sacked,H_Total Plays,H_Average Gain,H_Interceptions,H_Fumbles Lost,H_Penalty Avg. Yds Lost,H_Third Down Pct,H_Fourth Down Pct,Outcome
0,17,316,52,21,2.5,264,306,9.0,5,60,...,6.6,3,69,5.1,4,0,8.714286,0.33,0.0,1


In [35]:
combined

Unnamed: 0,First downs,Total Net Yards,Net Yards Rushing,Rushing Plays,Avg. Yds/Rush,Net Yards Passing,Gross Yards Passing,Avg. Yds/Att Passing,Sacked,Total Plays,Average Gain,Interceptions,Fumbles Lost,Penalty Avg. Yds Lost,Third Down Pct,Fourth Down Pct,Home,Outcome
TennesseeTen,17,316,52,21,2.5,264,306,9.0,5,60,5.3,0,0,7.222222,0.38,0.5,0,0
PittsburghPit,25,349,65,21,3.1,284,299,6.6,3,69,5.1,4,0,8.714286,0.33,0.0,1,1


In [36]:
combined.columns

Index(['First downs', 'Total Net Yards', 'Net Yards Rushing', 'Rushing Plays',
       'Avg. Yds/Rush', 'Net Yards Passing', 'Gross Yards Passing',
       'Avg. Yds/Att Passing', 'Sacked', 'Total Plays', 'Average Gain',
       'Interceptions', 'Fumbles Lost', 'Penalty Avg. Yds Lost',
       'Third Down Pct', 'Fourth Down Pct', 'Home', 'Outcome'],
      dtype='object', name='Unnamed: 0')

In [37]:
## adding team data to dictionary for each team

team_dict = {}

for i in combined.index:
    if i in team_dict:
        team_dict[i].append(combined.loc[i])
    else:
        team_dict[i] = [combined.loc[i]]

In [38]:
team_dict

{'TennesseeTen': [Unnamed: 0
  First downs                   17
  Total Net Yards              316
  Net Yards Rushing             52
  Rushing Plays                 21
  Avg. Yds/Rush                2.5
  Net Yards Passing            264
  Gross Yards Passing          306
  Avg. Yds/Att Passing         9.0
  Sacked                         5
  Total Plays                   60
  Average Gain                 5.3
  Interceptions                  0
  Fumbles Lost                   0
  Penalty Avg. Yds Lost    7.22222
  Third Down Pct              0.38
  Fourth Down Pct              0.5
  Home                           0
  Outcome                        0
  Name: TennesseeTen, dtype: object], 'PittsburghPit': [Unnamed: 0
  First downs                   25
  Total Net Yards              349
  Net Yards Rushing             65
  Rushing Plays                 21
  Avg. Yds/Rush                3.1
  Net Yards Passing            284
  Gross Yards Passing          299
  Avg. Yds/Att Passing       

In [39]:
## write a function to iterate over game ids and gather game data

def get_game_data(game_ids):
    
    all_games_df = pd.DataFrame()
    team_dict = {}
    
    for year in game_ids:
        week_counter = 1
        
        for week in year:
            for game in week:
                game_data = pd.read_html(f'https://www.footballdb.com/games/boxscore.html?gid={game}')
                score = game_data[0]
                data_1 = game_data[2].set_index('Unnamed: 0').T
                data_2 = game_data[3].set_index('Unnamed: 0').T

                data_1.drop(labels=['Rushing', 'Passing', 'Penalty', 'Att - Comp - Int'], axis=1, inplace=True)
                data_1['Sacked'] = [data_1['Sacked - Yds Lost'][0].split('-')[0], data_1['Sacked - Yds Lost'][1].split('-')[0]]
                data_1.drop('Sacked - Yds Lost', axis=1, inplace=True)
                data_1.rename(columns={'Avg. Yds/Att': 'Avg. Yds/Att Passing', 'Average Gain': 'Avg. Yds/Rush'}, inplace=True)

                data_2.drop(labels=['Punts - Average', 'Had Blocked', 'Punt Returns', 'Kickoff Returns', 'Field Goals', 'Time of Possession'], axis=1, inplace=True)
                data_2['Interceptions'] = [data_2['Interception Returns'][0].split('-')[0], data_2['Interception Returns'][1].split('-')[0]]
                data_2['Fumbles Lost'] = [data_2['Fumbles - Lost'][0].split('-')[1], data_2['Fumbles - Lost'][1].split('-')[1]]

                v_penalties = int(data_2['Penalties - Yards'][0].split('-')[0])
                h_penalties = int(data_2['Penalties - Yards'][1].split('-')[0])

                if v_penalties == 0 and h_penalties != 0:
                    data_2['Penalty Avg. Yds Lost'] = [0, int(data_2['Penalties - Yards'][1].split('-')[1]) / h_penalties]
                elif v_penalties != 0 and h_penalties == 0:
                    data_2['Penalty Avg. Yds Lost'] = [int(data_2['Penalties - Yards'][0].split('-')[1]) / v_penalties, 0]
                elif v_penalties == 0 and h_penalties == 0:
                    data_2['Penalty Avg. Yds Lost'] = [0, 0]
                else:
                    data_2['Penalty Avg. Yds Lost'] = [int(data_2['Penalties - Yards'][0].split('-')[1]) / v_penalties,
                                                       int(data_2['Penalties - Yards'][1].split('-')[1]) / h_penalties]

                data_2['Third Down Pct'] = [int(data_2['Third Downs'][0].split('-')[2][:-1])/100, int(data_2['Third Downs'][1].split('-')[2][:-1])/100]
                data_2['Fourth Down Pct'] = [int(data_2['Fourth Downs'][0].split('-')[2][:-1])/100, int(data_2['Fourth Downs'][1].split('-')[2][:-1])/100]
                data_2.drop(labels=['Interception Returns', 'Penalties - Yards', 'Fumbles - Lost', 'Third Downs', 'Fourth Downs'], axis=1, inplace=True)

                combined = pd.concat([data_1, data_2], axis=1)
                visitor_columns = []
                home_columns = []

                for column in combined.columns:
                    visitor_columns.append('V_' + column)
                    home_columns.append('H_' + column)

                game_data = list(combined.iloc[0]) + list(combined.iloc[1])
                pd.to_numeric(game_data)
                game_df = pd.DataFrame([game_data], columns=visitor_columns + home_columns)

                if int(score[5][1]) > int(score[5][2]):
                    game_df['Outcome'] = 0
                    combined['Outcome'] = [1, 0]
                elif int(score[5][1]) == int(score[5][2]):
                    game_df['Outcome'] = 0.5
                    combined['Outcome'] = [0.5, 0.5]
                else:
                    game_df['Outcome'] = 1
                    combined['Outcome'] = [0, 1]

                ## add home and week columns to combined df
                combined['Home'] = [0, 1]
                combined['Week'] = [week_counter, week_counter]

                all_games_df = all_games_df.append(game_df, ignore_index=True)

                for i in combined.index:
                    if i in team_dict:
                        team_dict[i].append(combined.loc[i])
                    else:
                        team_dict[i] = [combined.loc[i]]

                time.sleep(np.random.randint(5, 10))

            week_counter += 1
        
    return all_games_df, team_dict

In [46]:
## getting all game ids by year

year_2017 = get_game_ids('https://www.footballdb.com/games/index.html?lg=NFL&yr=2017') 
year_2018 = get_game_ids('https://www.footballdb.com/games/index.html?lg=NFL&yr=2018')
year_2019 = get_game_ids('https://www.footballdb.com/games/index.html?lg=NFL&yr=2019')

In [47]:
year_2017

[['2017090701',
  '2017091001',
  '2017091002',
  '2017091003',
  '2017091004',
  '2017091005',
  '2017091006',
  '2017091007',
  '2017091008',
  '2017091009',
  '2017091010',
  '2017091011',
  '2017091012',
  '2017091101',
  '2017091102'],
 ['2017091401',
  '2017091701',
  '2017091702',
  '2017091703',
  '2017091704',
  '2017091705',
  '2017091706',
  '2017091707',
  '2017091708',
  '2017091709',
  '2017091710',
  '2017091711',
  '2017091712',
  '2017091713',
  '2017091714',
  '2017091801'],
 ['2017092101',
  '2017092401',
  '2017092402',
  '2017092403',
  '2017092404',
  '2017092405',
  '2017092406',
  '2017092407',
  '2017092408',
  '2017092409',
  '2017092410',
  '2017092411',
  '2017092412',
  '2017092413',
  '2017092414',
  '2017092501'],
 ['2017092801',
  '2017100101',
  '2017100102',
  '2017100103',
  '2017100104',
  '2017100105',
  '2017100106',
  '2017100107',
  '2017100108',
  '2017100109',
  '2017100110',
  '2017100111',
  '2017100112',
  '2017100113',
  '2017100114',
  '20

In [48]:
## fixing game ids that are not accounted for correctly

year_2017[0][-4] = '2017091013'
year_2017[10][-2] = '2017091011'

year_2018[3][1] = '2018093014'
year_2018[4][-1] = '2018100803'
year_2018[7][1] = '2018102813'
year_2018[15][0] = '2018122314'
year_2018[15][1] = '2018122315'

year_2019[15][0] = '2019122213'
year_2019[15][1] = '2019122214'
year_2019[15][2] = '2019122215'

In [49]:
## combining all game ids into one list

all_game_ids = [year_2017] + [year_2018] + [year_2019]

In [50]:
all_games_df, team_dict = get_game_data(all_game_ids)

In [51]:
## checking to make sure function worked correctly

all_games_df

,V_First downs,V_Total Net Yards,V_Net Yards Rushing,V_Rushing Plays,V_Avg. Yds/Rush,V_Net Yards Passing,V_Gross Yards Passing,V_Avg. Yds/Att Passing,V_Sacked,V_Total Plays,...,H_Avg. Yds/Att Passing,H_Sacked,H_Total Plays,H_Average Gain,H_Interceptions,H_Fumbles Lost,H_Penalty Avg. Yds Lost,H_Third Down Pct,H_Fourth Down Pct,Outcome
0,26,537,185,27,6.9,352,368,10.5,3,65,...,7.4,3,74,5.0,0,0,9.166667,0.33,0.00,0.0
1,24,308,45,18,2.5,263,268,5.6,1,67,...,7.1,1,69,5.3,3,0,10.000000,0.50,0.00,1.0
2,18,372,64,23,2.8,308,321,10.7,2,55,...,5.3,4,63,4.8,0,0,10.000000,0.38,0.50,0.0
3,17,268,157,42,3.7,111,121,7.1,1,60,...,5.5,5,58,3.8,1,1,7.333333,0.30,0.00,0.0
4,11,214,38,15,2.5,176,187,4.8,1,55,...,8.0,2,72,5.7,2,0,8.333333,0.47,0.00,1.0
5,20,287,116,38,3.1,171,171,6.8,0,63,...,5.5,4,54,4.0,1,1,7.400000,0.18,0.25,0.0
6,16,290,35,17,2.1,255,263,7.3,1,54,...,7.4,7,62,3.8,1,0,15.250000,0.25,1.00,0.0
7,13,233,35,12,2.9,198,220,5.8,3,53,...,6.9,1,71,5.5,1,0,10.000000,0.53,0.00,1.0
8,12,225,90,18,5.0,135,158,5.9,3,48,...,7.4,4,74,5.0,0,0,8.750000,0.56,0.00,1.0
9,19,280,155,39,4.0,125,125,6.0,0,60,...,4.6,10,69,2.9,0,3,10.444444,0.41,0.00,0.0


In [59]:
## making a dataframe for each team

Kansas_City = pd.DataFrame(team_dict['Kansas CityKC'])
New_England = pd.DataFrame(team_dict['New EnglandNE'])
Arizona = pd.DataFrame(team_dict['ArizonaAri'])
Detroit = pd.DataFrame(team_dict['DetroitDet'])
Atlanta = pd.DataFrame(team_dict['AtlantaAtl'])
Chicago = pd.DataFrame(team_dict['ChicagoChi'])
Baltimore = pd.DataFrame(team_dict['BaltimoreBal'])
Cincinnati = pd.DataFrame(team_dict['CincinnatiCin'])
NY_Jets = pd.DataFrame(team_dict['NY JetsNYJ'])
Buffalo = pd.DataFrame(team_dict['BuffaloBuf'])
Carolina = pd.DataFrame(team_dict['CarolinaCar'])
San_Francisco = pd.DataFrame(team_dict['San FranciscoSF'])
Pittsburgh = pd.DataFrame(team_dict['PittsburghPit'])
Cleveland = pd.DataFrame(team_dict['ClevelandCle'])
NY_Giants = pd.DataFrame(team_dict['NY GiantsNYG'])
Dallas = pd.DataFrame(team_dict['DallasDal'])
Seattle = pd.DataFrame(team_dict['SeattleSea'])
Green_Bay = pd.DataFrame(team_dict['Green BayGB'])
Jacksonville = pd.DataFrame(team_dict['JacksonvilleJax'])
Houston = pd.DataFrame(team_dict['HoustonHou'])
Indianapolis = pd.DataFrame(team_dict['IndianapolisInd'])
LA_Rams = pd.DataFrame(team_dict['LA RamsLA'])
Tampa_Bay = pd.DataFrame(team_dict['Tampa BayTB'])
Miami = pd.DataFrame(team_dict['MiamiMia'])
Oakland = pd.DataFrame(team_dict['OaklandOak'])
Tennessee = pd.DataFrame(team_dict['TennesseeTen'])
LA_Chargers = pd.DataFrame(team_dict['LA ChargersLAC'])
Denver = pd.DataFrame(team_dict['DenverDen'])
New_Orleans = pd.DataFrame(team_dict['New OrleansNO'])
Minnesota = pd.DataFrame(team_dict['MinnesotaMin'])
Philadelphia = pd.DataFrame(team_dict['PhiladelphiaPhi'])
Washington = pd.DataFrame(team_dict['WashingtonWas'])

In [60]:
list_of_teams = [Kansas_City, New_England, Arizona, Detroit, Atlanta, Chicago, Baltimore, Cincinnati, 
                 NY_Jets, Buffalo, Carolina, San_Francisco, Pittsburgh, Cleveland, NY_Giants, Dallas, 
                 Seattle, Green_Bay, Jacksonville, Houston, Indianapolis, LA_Rams, Philadelphia, Washington, 
                 Oakland, Tennessee, LA_Chargers, Denver, New_Orleans, Minnesota, Tampa_Bay, Miami]

In [61]:
team_names = list(team_dict.keys())

In [62]:
team_names

['Kansas CityKC',
 'New EnglandNE',
 'ArizonaAri',
 'DetroitDet',
 'AtlantaAtl',
 'ChicagoChi',
 'BaltimoreBal',
 'CincinnatiCin',
 'NY JetsNYJ',
 'BuffaloBuf',
 'CarolinaCar',
 'San FranciscoSF',
 'PittsburghPit',
 'ClevelandCle',
 'NY GiantsNYG',
 'DallasDal',
 'SeattleSea',
 'Green BayGB',
 'JacksonvilleJax',
 'HoustonHou',
 'IndianapolisInd',
 'LA RamsLA',
 'PhiladelphiaPhi',
 'WashingtonWas',
 'OaklandOak',
 'TennesseeTen',
 'LA ChargersLAC',
 'DenverDen',
 'New OrleansNO',
 'MinnesotaMin',
 'Tampa BayTB',
 'MiamiMia']

In [63]:
## exporting team dataframes/all games dataframe to csv files

for num in range(len(list_of_teams)):
    list_of_teams[num].to_csv(r'C:\Users\Chris\CourseMaterials\Capstone\dsc-capstone-project-v2-online-ds-pt-090919\data\{}.csv'.format(team_names[num]))

In [64]:
all_games_df.to_csv(r'C:\Users\Chris\CourseMaterials\Capstone\dsc-capstone-project-v2-online-ds-pt-090919\data\Games.csv')